In [46]:
import torch as tt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.ToTensor()
path = "./CIFAR-10"
train_dataset = datasets.CIFAR10(root=path, train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

print(len(train_dataset))

50000


In [47]:
def relu(x):
    return tt.maximum(x, tt.tensor(0.0, device=x.device))

def softmax(x):
    exp_x = tt.exp(x - x.max(dim=1, keepdim=True).values)
    return exp_x / exp_x.sum(dim=1, keepdim=True)

def cross_entropy(pred, target):
    N = target.size(0)
    log_likelihood = -tt.log(pred[tt.arange(N), target] + 1e-9)
    return log_likelihood.mean()

In [48]:
D_in = 3*32*32
h1, h2, h3, h4 = 1024, 512, 256, 128
D_out = 10

class Net(tt.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = tt.nn.Linear(D_in, h1)
        self.fc2 = tt.nn.Linear(h1, h2)
        self.fc3 = tt.nn.Linear(h2, h3)
        self.fc4 = tt.nn.Linear(h3, h4)
        self.fc5 = tt.nn.Linear(h4, D_out)

    def forward(self, x):
        x = x.view(x.size(0), -1)   # flatten
        x = relu(self.fc1(x))
        x = relu(self.fc2(x))
        x = relu(self.fc3(x))
        x = relu(self.fc4(x))
        x = self.fc5(x)
        return softmax(x)

In [ ]:
device = "cpu"
model = Net().to(device)
opt = tt.optim.Adam(model.parameters(), lr=1e-4)

epoch = 5
for _ in range(epoch):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)

        probs = model(imgs)
        loss = cross_entropy(probs, labels)

        opt.zero_grad()
        loss.backward()
        opt.step()

        if i % 100 == 0:
            print(f"Batch {i}, Loss: {loss.item():.4f}")

Batch 0, Loss: 2.3064
Batch 100, Loss: 2.1514
Batch 200, Loss: 2.0187
Batch 300, Loss: 2.0176
Batch 400, Loss: 1.7878
Batch 500, Loss: 1.9215
Batch 600, Loss: 1.8834
Batch 700, Loss: 1.9542
Batch 0, Loss: 1.8438
Batch 100, Loss: 1.7234
Batch 200, Loss: 1.8887
Batch 300, Loss: 1.6364
Batch 400, Loss: 1.5203
Batch 500, Loss: 1.8384
Batch 600, Loss: 1.6621
Batch 700, Loss: 1.7592
Batch 0, Loss: 1.6240
Batch 100, Loss: 1.7350
Batch 200, Loss: 1.5092
Batch 300, Loss: 1.5377
Batch 400, Loss: 1.5145
Batch 500, Loss: 1.6459
Batch 600, Loss: 1.7428
Batch 700, Loss: 1.6654
Batch 0, Loss: 1.7201
Batch 100, Loss: 1.5616
Batch 200, Loss: 1.5296
Batch 300, Loss: 1.6744
Batch 400, Loss: 1.6790
Batch 500, Loss: 1.6926
Batch 600, Loss: 1.6714
Batch 700, Loss: 1.6308
Batch 0, Loss: 1.7905
Batch 100, Loss: 1.5617
Batch 200, Loss: 1.5810
Batch 300, Loss: 1.7146
Batch 400, Loss: 1.4247
Batch 500, Loss: 1.4292
Batch 600, Loss: 1.5519
Batch 700, Loss: 1.4702


In [52]:
test_dataset = datasets.CIFAR10(root="./CIFAR-10", train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

def evaluate(model, loader):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with tt.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            probs = model(imgs)
            loss = cross_entropy(probs, labels)
            total_loss += loss.item() * imgs.size(0)
            preds = probs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    avg_loss = total_loss / total
    acc = correct / total
    return avg_loss, acc

test_loss, test_acc = evaluate(model, test_loader)
print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc*100:.2f}%")


Test Loss: 1.5219, Test Acc: 46.19%
